In [1]:
import utilities
import models
import torch
import importlib
importlib.reload(models)
#importlib.reload(escnn)


<module 'models' from '/home/kaitlin/repos/equiNAS/models.py'>

In [32]:
model = models.TDEquiCNN(gs = [(1,3) for _ in range(2)]+[(1,2) for _ in range(4)])


In [43]:
print(type(model.blocks[0]._modules['0']))
for i in range(1,2):
    print(len(model.blocks[i]._modules['0'].in_type.representations))
    print(model.blocks[i]._modules['0'].weights.shape)
    print(model.blocks[i]._modules['0'].filter.shape)
    print(model.blocks[i]._modules['0'].space.build_kernel_basis)
    print(model.blocks[i]._modules['0'].space)
    print(model.blocks[i]._modules['0'].basisexpansion._n_pairs)
    for key, value in model.blocks[i]._modules['0'].basisexpansion.__dict__.items():
        print(key, value)
    #print(getattr(model.blocks[i]._modules['0'].basisexpansion, "out_indices_('regular', 'regular')"))


<class 'escnn.nn.modules.conv.r2convolution.R2Conv'>
8
torch.Size([11264])
torch.Size([128, 128, 5, 5])
<bound method GSpace.build_kernel_basis of D8_on_R2[(3.141592653589793, 8)]>
D8_on_R2[(3.141592653589793, 8)]
1
training True
_parameters OrderedDict()
_buffers OrderedDict()
_non_persistent_buffers_set set()
_backward_hooks OrderedDict()
_is_full_backward_hook None
_forward_hooks OrderedDict()
_forward_pre_hooks OrderedDict()
_state_dict_hooks OrderedDict()
_load_state_dict_pre_hooks OrderedDict()
_modules OrderedDict([("block_expansion_('regular', 'regular')", SingleBlockBasisExpansion())])
_in_reprs [D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16]
_out_reprs [D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16, D8|[regular]:16]
_input_size 128
_output_size 128
points [[-2. -1.  0.  1.  2. -2. -1.  0.  1.  2. -2. -1.  0.  1.  2. -

In [37]:
model(torch.randn(2,1,29,29))

tensor([[ 0.5413, -0.1534, -0.4405,  0.6337, -0.5429,  0.7636, -0.3445,  1.0023,
         -0.0882, -0.2319],
        [-0.1932,  0.4594,  0.6599, -0.3193,  0.2874, -0.9569,  0.4328, -0.7026,
          0.1392,  0.3315]], grad_fn=<AddmmBackward0>)

In [ ]:
train_loader, validation_loader, test_loader = utilities.get_dataloaders(path_to_dir="..")


In [2]:
model = models.TDRegEquiCNN(gs = [(0,2) for _ in range(6)]+[(0,0) for _ in range(0)])
